## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**3421. Find Students Who Improved (Medium)**

**Table: Scores**

| Column Name | Type    |
|-------------|---------|
| student_id  | int     |
| subject     | varchar |
| score       | int     |
| exam_date   | varchar |

(student_id, subject, exam_date) is the primary key for this table.
Each row contains information about a student's score in a specific subject on a particular exam date. score is between 0 and 100 (inclusive).

**Write a solution to find the students who have shown improvement. A student is considered to have shown improvement if they meet both of these conditions:**

- Have taken exams in the same subject on at least two different dates
- Their latest score in that subject is higher than their first score

Return the result table ordered by student_id, subject in ascending order.

The result format is in the following example.

**Example:**

**Input:**

**Scores table:**

| student_id | subject  | score | exam_date  |
|------------|----------|-------|------------|
| 101        | Math     | 70    | 2023-01-15 |
| 101        | Math     | 85    | 2023-02-15 |
| 101        | Physics  | 65    | 2023-01-15 |
| 101        | Physics  | 60    | 2023-02-15 |
| 102        | Math     | 80    | 2023-01-15 |
| 102        | Math     | 85    | 2023-02-15 |
| 103        | Math     | 90    | 2023-01-15 |
| 104        | Physics  | 75    | 2023-01-15 |
| 104        | Physics  | 85    | 2023-02-15 |

**Output:**

| student_id | subject  | first_score | latest_score |
|------------|----------|-------------|--------------|
| 101        | Math     | 70          | 85           |
| 102        | Math     | 80          | 85           |
| 104        | Physics  | 75          | 85           |

**Explanation:**

- Student 101 in Math: Improved from 70 to 85
- Student 101 in Physics: No improvement (dropped from 65 to 60)
- Student 102 in Math: Improved from 80 to 85
- Student 103 in Math: Only one exam, not eligible
- Student 104 in Physics: Improved from 75 to 85

Result table is ordered by student_id, subject.

In [0]:
scores_data_3421 = [
    (101, "Math", 70, "2023-01-15"),
    (101, "Math", 85, "2023-02-15"),
    (101, "Physics", 65, "2023-01-15"),
    (101, "Physics", 60, "2023-02-15"),
    (102, "Math", 80, "2023-01-15"),
    (102, "Math", 85, "2023-02-15"),
    (103, "Math", 90, "2023-01-15"),
    (104, "Physics", 75, "2023-01-15"),
    (104, "Physics", 85, "2023-02-15")
]

scores_columns_3421 = ["student_id", "subject", "score", "exam_date"]
scores_df_3421 = spark.createDataFrame(scores_data_3421, scores_columns_3421)
scores_df_3421.show()

+----------+-------+-----+----------+
|student_id|subject|score| exam_date|
+----------+-------+-----+----------+
|       101|   Math|   70|2023-01-15|
|       101|   Math|   85|2023-02-15|
|       101|Physics|   65|2023-01-15|
|       101|Physics|   60|2023-02-15|
|       102|   Math|   80|2023-01-15|
|       102|   Math|   85|2023-02-15|
|       103|   Math|   90|2023-01-15|
|       104|Physics|   75|2023-01-15|
|       104|Physics|   85|2023-02-15|
+----------+-------+-----+----------+



In [0]:
w_asc = Window.partitionBy("student_id", "subject").orderBy("exam_date")
w_desc = Window.partitionBy("student_id", "subject").orderBy(col("exam_date").desc())

In [0]:
first_score_df_3421 = scores_df_3421\
                        .withColumn("rn", row_number().over(w_asc)) \
                            .filter(col("rn") == 1) \
                                .select("student_id", "subject", col("score").alias("first_score"))

In [0]:
latest_score_df_3421 = scores_df_3421\
                        .withColumn("rn", row_number().over(w_desc)) \
                            .filter(col("rn") == 1) \
                                .select("student_id", "subject", col("score").alias("latest_score"))

In [0]:
first_score_df_3421\
    .join(latest_score_df_3421, ["student_id", "subject"]) \
        .filter(col("latest_score") > col("first_score")) \
            .orderBy("student_id", "subject").display()

student_id,subject,first_score,latest_score
101,Math,70,85
102,Math,80,85
104,Physics,75,85
